In [11]:
import sys

import pandas as pd
import numpy as np
import time
# import turicreate as tc
from sklearn.cross_validation import train_test_split

ModuleNotFoundError: No module named 'sklearn.cross_validation'

In [24]:
transaction = pd.read_csv('transaction.csv')

customers = transaction[['settop_id','gender']]
trx = transaction[['settop_id', 'prod_id', 'order_count', 'purchase_dt']]
transaction.head()

,purchase_dt,uid,gender,prod_id,prod_name,order_count,cancle_count,order_id,settop_id
0,2020-10-09 오후 6:59:14,201708425482,여,10175535,숀리 다이어트킹 스쿼트머신,1,0,20201009992653,0010427579
1,2020-10-13 오후 1:30:46,201708425482,여,10225923,직세일 한경희 건강식 마스터 죽제조기,1,0,20201013100206,0010427579
2,2020-10-28 오후 12:46:03,201707295540,여,10229168,코코스윗 키즈 남아 10종,1,0,20201028382282,001632d78daa
3,2020-10-04 오후 4:52:18,201712694758,남,10160229,흥국화재파워라이프종합보험20.06_신세계상품권,1,0,20201004715571,001632fb6360
4,2020-10-24 오후 12:16:10,201706321266,여,10208586,20FW리더잭 여성웜기모밴딩데님3종(F453),1,0,20201024898004,001632fddf03


In [19]:
customers.head()

,settop_id,gender
0,0010427579,여
1,0010427579,여
2,001632d78daa,여
3,001632fb6360,남
4,001632fddf03,여


In [25]:
trx.head()

,settop_id,prod_id,order_count,purchase_dt
0,0010427579,10175535,1,2020-10-09 오후 6:59:14
1,0010427579,10225923,1,2020-10-13 오후 1:30:46
2,001632d78daa,10229168,1,2020-10-28 오후 12:46:03
3,001632fb6360,10160229,1,2020-10-04 오후 4:52:18
4,001632fddf03,10208586,1,2020-10-24 오후 12:16:10


In [28]:
data = pd.melt(trx.set_index('settop_id')['prod_id'].apply(pd.Series).reset_index(), 
             id_vars=['settop_id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['settop_id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'order_count'}) \
    .reset_index() \
    .rename(columns={'products': 'prod_id'})
data['prod_id'] = data['prod_id'].astype(np.int64)

In [29]:
data.head()

,settop_id,prod_id,order_count
0,000029EC-8BAC-11DE-B0D3-BB8606F0C478,10103773,1
1,000029EC-8BAC-11DE-B0D3-BB8606F0C478,10123399,1
2,000029EC-8BAC-11DE-B0D3-BB8606F0C478,10132254,1
3,000029EC-8BAC-11DE-B0D3-BB8606F0C478,10132255,1
4,00003389-7F49-11E3-BD90-577F9A9EAEE5,10093197,1


In [30]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)

In [31]:
df_matrix = pd.pivot_table(data, values='order_count', index='settop_id', columns='prod_id')

In [33]:
df_matrix.head()

prod_id,10000148,10000499,10000823,10001064,10001342,10002505,10002518,10002711,10002869,10003991,...,10244227,10244327,10244328,10244349,10244595,10244622,10245186,10245321,10246451,10246870
settop_id,,,,,,,,,,,,,,,,,,,,,
000029EC-8BAC-11DE-B0D3-BB8606F0C478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00003389-7F49-11E3-BD90-577F9A9EAEE5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00097154-FAB9-11E4-B376-51F48D49EFCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000F8E38-C42C-11E1-8F8A-5353790B8755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0010408978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df_matrix_norm = (df_matrix - df_matrix.min()) / (df_matrix.max() - df_matrix.min())

In [34]:
df_matrix_norm.head()

prod_id,10000148,10000499,10000823,10001064,10001342,10002505,10002518,10002711,10002869,10003991,...,10244227,10244327,10244328,10244349,10244595,10244622,10245186,10245321,10246451,10246870
settop_id,,,,,,,,,,,,,,,,,,,,,
000029EC-8BAC-11DE-B0D3-BB8606F0C478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00003389-7F49-11E3-BD90-577F9A9EAEE5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00097154-FAB9-11E4-B376-51F48D49EFCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000F8E38-C42C-11E1-8F8A-5353790B8755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0010408978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


create a table for input to the modeling

In [35]:
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['settop_id'], value_name='scaled_purchased_freq').dropna()

The above steps can be combined to a function defined below:


In [ ]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='order_count', index='settopd_id', columns='prod_id')
    df_matrix_norm = (df_matrix - df_matrix.min()) / (df_matrix.max() - df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']

    return pd.melt(d, id_vars=['settop_id'], value_name='scaled_purchase_freq').dropna()

In [ ]:
def split_data(data):
    """
    Splits dataset into training and test set.

    Args:
        data (pandas.DataFrame)
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    """
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [ ]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = slit_data(data_norm)